In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
from cerberus.cerberus import *

## end aggregation config file parsing

In [41]:
def parse_agg_ends_config(fname):
    """
    Parse a config file for end aggregation
    
    Parameters:
        fname (str): Path to the tss / tes config file
    
    Returns:
        beds (list of str): List of fnames to beds
        add_ends (list of bool): List of whether or not
            to add new ends from each bed
        sources (list of str): Source names for each bed
    """
    df = pd.read_csv(fname, header=None, sep=',')
    df.columns = ['fname', 'add_ends', 'source']
    
    beds = df.fname.tolist()
    add_ends = df.add_ends.tolist()
    sources = df.source.tolist()
    
    return beds, add_ends, sources
    

In [43]:
beds, add_ends, sources = parse_agg_ends_config(fname)
print(beds)
print(add_ends)
print(sources)

['/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/v39_tss.bed', '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/v29_tss.bed', '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/test_no_gid_no_strand.bed', '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/talon_tss.bed']
[True, True, False, True]
['v39', 'v29', 'ccre', 'talon']


In [37]:
d = '/Users/fairliereese/Documents/programming/mortazavi_lab/data/rnawg/lr_bulk/cerberus/'
v29_bed = '{}v29_tss.bed'.format(d)
v39_bed = '{}v39_tss.bed'.format(d)
talon_bed = '{}talon_tss.bed'.format(d)
no_gid = '{}test_no_gid_no_strand.bed'.format(d)

# beds = [v39_bed, v29_bed, talon_bed] 
# beds = [v29_bed, talon_bed, v39_bed] 
beds = [v39_bed, v29_bed, no_gid, talon_bed] 
sources = ['v39', 'v29', 'ccre', 'talon']
add_ends = [True, True, False, True]

df = pd.DataFrame()
df['fname'] = beds
df['add_ends'] = add_ends
df['source'] = sources

df.head()
fname = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/agg_tss_config.csv'
df.to_csv(fname, sep=',', index=False, header=None)

In [35]:


# mode = 'tss'
# slack = 20



# df = aggregate_ends(beds, sources, add_ends, slack, mode)

# # fname = 'tests/files/Canx_1_tss.bed'
# # df = read_bed(fname, mode='tss')

# # fname = 'tests/files/Canx_ccre_pels.bed'
# # # df = read_bed(fname, mode='tss')